# House Price Prediction
**Workspace for the [Machine Learning course](https://www.kaggle.com/learn/machine-learning).**


Reading the csv files and printing basic details about the data

In [6]:
import pandas as pd

pd.set_option('display.max_rows', 5)
# main_file_path = '../input/house-prices-advanced-regression-techniques/train.csv' # this is the path to the Iowa data that you will use
#test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
test = pd.read_csv('test.csv')
main_file_path = "train.csv"
data = pd.read_csv(main_file_path)

print(data.describe())
print(data.head())
print(data.columns)

            Id  MSSubClass  LotFrontage        LotArea  OverallQual  \
count  1460.00  1460.00000  1201.000000    1460.000000  1460.000000   
mean    730.50    56.89726    70.049958   10516.828082     6.099315   
...        ...         ...          ...            ...          ...   
75%    1095.25    70.00000    80.000000   11601.500000     7.000000   
max    1460.00   190.00000   313.000000  215245.000000    10.000000   

       OverallCond    YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1  \
count  1460.000000  1460.000000   1460.000000  1452.000000  1460.000000   
mean      5.575342  1971.267808   1984.865753   103.685262   443.639726   
...            ...          ...           ...          ...          ...   
75%       6.000000  2000.000000   2004.000000   166.000000   712.250000   
max       9.000000  2010.000000   2010.000000  1600.000000  5644.000000   

           ...        WoodDeckSF  OpenPorchSF  EnclosedPorch    3SsnPorch  \
count      ...       1460.000000  1460.000000

**Checking Sale Price Column**

In [7]:
print(data["SalePrice"].describe())
print(data["SalePrice"].head())

count      1460.00000
mean     180921.19589
             ...     
75%      214000.00000
max      755000.00000
Name: SalePrice, Length: 8, dtype: float64
0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64


# Feature Selection

In [8]:
feature_list = ["LotArea","YearBuilt","1stFlrSF","2ndFlrSF","FullBath","BedroomAbvGr","TotRmsAbvGrd"]
X = data[feature_list]
y = data["SalePrice"]
X.head()
y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

# Decision Tree

In [9]:
from sklearn.tree import DecisionTreeRegressor as dt

iowa_model = dt()
iowa_model.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [10]:
iowa_model.predict(X.head())

array([ 208500.,  181500.,  223500.,  140000.,  250000.])

**Checking MAE using built-in method**

In [11]:
from sklearn.metrics import mean_absolute_error

predicted_home_prices = iowa_model.predict(X)
mean_absolute_error(y, predicted_home_prices)

62.354337899543388

**Splitting into test-train sets**

In [13]:
#split training and validation data using scikit-learns inbuilt function

from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)

iowa_model.fit(train_X,train_y)
predicted_home_prices = iowa_model.predict(val_X)
mean_absolute_error(val_y,predicted_home_prices)

33474.109589041094

**Searching for optimal leaf nodes**

In [14]:
def get_mae(max_leaf_nodes, predictors_train, predictors_val, targ_train, targ_val):
    model = dt(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(predictors_train, targ_train)
    preds_val = model.predict(predictors_val)
    mae = mean_absolute_error(targ_val, preds_val)
    return(mae)

for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  35190
Max leaf nodes: 50  		 Mean Absolute Error:  27825
Max leaf nodes: 500  		 Mean Absolute Error:  32662
Max leaf nodes: 5000  		 Mean Absolute Error:  33382


# Ramdom Forest

In [18]:
from sklearn.ensemble import RandomForestRegressor

forest_model = RandomForestRegressor()
forest_model.fit(train_X, train_y)
melb_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, melb_preds))

24875.4865023


In [19]:
test_features = test[feature_list]
predicted_prices = forest_model.predict(test_features)
print(predicted_prices)

[ 132165.  160290.  182350. ...,  140990.  128350.  218941.]


In [20]:
#creat submission file called submission.csv
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice': predicted_prices})
my_submission.to_csv('submission.csv', index=False)

# Handling Missing Values

**Drop Columns with Missing Values**

In [44]:
data_without_missing_values = data.dropna(axis=1)

cols_with_missing = [col for col in data.columns if data[col].isnull().any()]
reduced_X_train = train_X.drop(cols_with_missing, axis=1)
reduced_X_test  = val_X.drop(cols_with_missing, axis=1)

print("Mean Absolute Error from dropping columns with Missing Values:")
print(score_dataset(reduced_X_train, reduced_X_test, y_train, y_test))

ValueError: labels ['LotFrontage' 'Alley' 'MasVnrType' 'MasVnrArea' 'BsmtQual' 'BsmtCond'
 'BsmtExposure' 'BsmtFinType1' 'BsmtFinType2' 'Electrical' 'FireplaceQu'
 'GarageType' 'GarageYrBlt' 'GarageFinish' 'GarageQual' 'GarageCond'
 'PoolQC' 'Fence' 'MiscFeature'] not contained in axis

**Imputation**

Imputation fills in the missing value with some number. The default behavior fills in the mean value for imputation. Statisticians have researched more complex strategies, but those complex strategies typically give no benefit once you plug the results into sophisticated machine learning models.

One (of many) nice things about Imputation is that it can be included in a scikit-learn Pipeline. Pipelines simplify model building, model validation and model deployment.

In [41]:
from sklearn.preprocessing import Imputer

def score_dataset(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return mean_absolute_error(y_test, preds)

my_imputer = Imputer()
imputed_X_train = my_imputer.fit_transform(train_X)
imputed_X_test = my_imputer.transform(val_X)
print("Mean Absolute Error from Imputation:")
print(score_dataset(imputed_X_train, imputed_X_test, train_y, val_y))

Mean Absolute Error from Imputation:
23676.2241553


In [34]:
data

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0        1          60       RL         65.0     8450   Pave   NaN      Reg   
1        2          20       RL         80.0     9600   Pave   NaN      Reg   
...    ...         ...      ...          ...      ...    ...   ...      ...   
1458  1459          20       RL         68.0     9717   Pave   NaN      Reg   
1459  1460          20       RL         75.0     9937   Pave   NaN      Reg   

     LandContour Utilities    ...     PoolArea PoolQC Fence MiscFeature  \
0            Lvl    AllPub    ...            0    NaN   NaN         NaN   
1            Lvl    AllPub    ...            0    NaN   NaN         NaN   
...          ...       ...    ...          ...    ...   ...         ...   
1458         Lvl    AllPub    ...            0    NaN   NaN         NaN   
1459         Lvl    AllPub    ...            0    NaN   NaN         NaN   

     MiscVal MoSold YrSold  SaleType  SaleCondition  SalePrice  
0          0      2   2008        WD         Normal     208500  
1          0      5   2007        WD         Normal     181500  
...      ...    ...    ...       ...            ...        ...  
1458       0      4   2010        WD         Normal     142125  
1459       0      6   2008        WD         Normal     147500  

[1460 rows x 81 columns]